# HW2 - Amit Parizat & Yoel Melul

## Theory Questions

### Q1

To evaluate how well our model performs at T1D classification, we need to have evaluation metrics that measures of its performances/accuracy. Which evaluation metric is more important to us: model accuracy or model performance? Give a simple example that illustrates your claim.

In our model the more important evaluation metric is